### Get data for training BERT

In [ ]:
data_ver = input("Choose version dataset of SQuAD : ")
print(f"-> set train,dev is {data_ver}")
if(data_ver == "v1"):
    test_url,train_url =  \
    "https://raw.githubusercontent.com/Purushothaman-natarajan/Q-and-A-chat-bot-from-PDF/main/SQuAD%20%20train%20data%20set/dev-v1.1.json",\
    "https://raw.githubusercontent.com/Purushothaman-natarajan/Q-and-A-chat-bot-from-PDF/main/SQuAD%20%20train%20data%20set/train-v1.1.json"
else:
    test_url,train_url =  \
    "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json",\
    "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json",


Choose version dataset of SQuAD : v2
-> set train,dev is v2


In [ ]:
import requests
import json

r = requests.get(train_url)
train_data = r.json()
r = requests.get(test_url)
test_data = r.json()

with open("/content/train-v1.1.json", "w") as file:
    json.dump(train_data, file)
with open("/content/dev-v1.1.json", "w") as file:
    json.dump(test_data, file)

### Set up dependencies

In [ ]:
!pip install -q nltk
!pip install -q autocorrect
!pip install -q transformers
!pip install -q gradio
!pip install -q beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import tensorflow as tf
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
from bs4 import BeautifulSoup
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.corpus import stopwords, brown
stop_words  = stopwords.words('english')
from autocorrect import Speller
from nltk.tokenize import word_tokenize
import requests
import json
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
import torch

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Pre-processing

In [ ]:
#extracting the url
def extract_text_from_url(url):
    print("---get crawl data---")
    response = requests.get(url=url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'lxml')
    paragraphs = soup.find_all('p')
    text_crawled = ' .  '.join([p.get_text(strip=True) for p in paragraphs])


    return text_crawled

In [ ]:
def clean_text_basic(text):
    # Remove square brackets and their contents
    cleaned_text = re.sub(r'\[.*?\]', '', text)

    # Remove leading and trailing whitespace
    cleaned_text = cleaned_text.strip()

    return cleaned_text
#lower casing the text
def to_lower(text):
    """
    Converting text to lower case as in, converting "Hello" to  "hello" or "Hi" to "hi".
    """
    # Specll check the words
    spell  = Speller(lang='en')
    texts = spell(text)
    return ' '.join([w.lower() for w in word_tokenize(text)])


In [ ]:
url = "https://www.britannica.com/science/solar-system"
text = extract_text_from_url(url)

In [ ]:
text = clean_text_basic(text)

In [ ]:
text[10000:100000]

'nward kuiper belt interplanetary mediuman exceedingly tenuous plasma ionized gas laced concentrations dust particlesextends outward sun au special offer students finish semester strong britannica learn solar system even contains objects interstellar space passing two interstellar objects observed oumuamua unusual cigarlike pancakelike shape possibly composed nitrogen ice comet borisov much like comets solar system much higher abundance carbon monoxide orbits solar system scalethe eight planets solar system pluto montage images scaled show approximate sizes bodies relative one another outward sun represented scale yellow segment extreme left four rocky terrestrial planets mercury venus earth mars four hydrogenrich giant planets jupiter saturn uranus neptune icy comparatively tiny plutomoreall planets dwarf planets rocky asteroids icy bodies kuiper belt move around sun elliptical orbits direction sun rotates motion termed prograde direct motion looking system vantage point earths north 

In [ ]:
# Load the SQuAD dataset JSON file
# def file_get_contents(fname):
#     if s.stripos(fname, 'http://'):
#         import urllib2
#         return urllib2.urlopen(fname).read(maxUrlRead)
#     else:
#         return open(fname).read(maxFileRead)

#Training data
with open(r"/content/train-v1.1.json", 'r') as f:
    squad_data = json.load(f)

training_data = []
for article in squad_data['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            if qa['answers']:
                answer = qa['answers'][0]['text']
                training_data.append({'question': question, 'answer': answer, 'context': context})

# Save the processed training data in a new JSON file
with open('training_data.json', 'w') as f:
    json.dump(training_data, f, indent=4)

print("Training data preparation complete.")

Training data preparation complete.


In [ ]:
training_data[:15]

[{'question': 'When did Beyonce start becoming popular?',
  'answer': 'in the late 1990s',
  'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'},
 {'question': 'What areas did Beyonce compete in when she was growing up?',
  'answer': 'singing and dancing',
  'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born Septe

In [ ]:
# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

# Prepare and tokenize your training data
# Format: [{"context": "...", "question": "...", "answer": "..."}, ...]
train_data = training_data[:15]  # Replace with your training data

# Train the model
optimizer = AdamW(model.parameters(), lr=2e-5)  # Adjust learning rate
model.train()

epochs = 10

for epoch in range(epochs):
    total_loss = 0
    for example in train_data:
        context = example['context']
        question = example['question']
        answer = example['answer']

        # Tokenize the context and question
        encoded_data = tokenizer(context, question, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

        # Find the positions of the answer in the context
        start_idx = context.find(answer)
        end_idx = start_idx + len(answer)

        if start_idx == -1:  # Handle cases where answer is not found
            continue

        # Prepare the inputs with the answer token positions
        inputs = {
            'input_ids': encoded_data['input_ids'],
            'attention_mask': encoded_data['attention_mask'],
            'start_positions': torch.tensor([start_idx]),
            'end_positions': torch.tensor([end_idx])
        }

        optimizer.zero_grad()
        outputs = model(**inputs)

        # Check if the loss is valid (not NaN or inf)
        loss = outputs.loss
        if loss != loss:
            continue  # Skip this batch if loss is NaN

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Save the trained model
model.save_pretrained("trained_qa_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 56.2482
Epoch 2, Loss: 53.5219
Epoch 3, Loss: 50.7598
Epoch 4, Loss: 46.6219
Epoch 5, Loss: 43.1953
Epoch 6, Loss: 41.6886
Epoch 7, Loss: 34.8387
Epoch 8, Loss: 35.7349
Epoch 9, Loss: 32.6585
Epoch 10, Loss: 24.2254


In [ ]:
def get_answer(context, question, model_path="trained_qa_model", max_seq_length=512):
    # Load the trained model and tokenizer
    model = BertForQuestionAnswering.from_pretrained(model_path)
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  # Same tokenizer as training

    # Tokenize the input question and context
    encoded_data = tokenizer.encode_plus(
        question,
        context,
        max_length=max_seq_length,
        truncation='only_second',  # Choose 'only_first' or 'only_second' to handle overflow
        padding='max_length',
        return_tensors='pt',
    )

    # Get the model's predicted start and end positions
    outputs = model(**encoded_data)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Find the token positions with the maximum probabilities
    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)

    # Retrieve the tokenized answer span
    answer_tokens = encoded_data['input_ids'][0][start_idx:end_idx+1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    return answer

### Test before give context

In [ ]:
question = 'who is Messi?'
predicted_answer = get_answer(context, question)
print("Predicted Answer:", predicted_answer)   #will return blank space if nothing is predicted

Predicted Answer: 


In [ ]:
question = 'What does AON stand for?'
predicted_answer = get_answer(context, question)
print("Predicted Answer:", predicted_answer)   #will return blank space if nothing is predicted

Predicted Answer: 


### Test after give context

In [ ]:
from transformers import pipeline

nlp = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", tokenizer="distilbert-base-cased")

context = text
question = "The solar system even contains objects from ?"

# pipeline to answer the question
result = nlp(question=question, context=context, padding="max_length", truncation=True)

print("Answer:", result['answer'])

Answer: interstellar space


In [ ]:
context = text

question = "What  are remnants of the planet-building process in the inner and outer solar system ? "
# pipeline to answer the question
result = nlp(question=question, context=context, padding="max_length", truncation=True)

print("Answer:", result['answer'])

Answer: rocky debris


## Test other url

#### Wiki 🌍

In [ ]:
url = r"https://en.wikipedia.org/wiki/Solar_System"
text = extract_text_from_url(url)
text = clean_text_basic(text)

---get crawl data---


In [ ]:
text

'.  TheSolar Systemis thegravitationallybound system of theSunand the objects that orbit it.It wasformed 4.6 billion years agowhen a dense region of amolecular cloudcollapsed, forming the Sun and aprotoplanetary disc. The Sun is an ordinarymain sequence starthat maintains abalanced equilibriumby thefusionof hydrogen into helium at itscore, releasing this energy from its outerphotosphere. .  The largest objects that orbit the Sun are the eightplanets. In order from the Sun, they are fourterrestrial planets(Mercury,Venus,EarthandMars); twogas giants(JupiterandSaturn); and twoice giants(UranusandNeptune). All terrestrial planets have solid surfaces. Inversely, allgiant planetsdo not have a definite surface, as they are mainly composed ofgasesandliquids. Over 99.86% of the Solar System\'s mass is in the Sun and nearly 90% of the remaining mass is in Jupiter and Saturn. .  There is a strong consensus among astronomersthat the Solar System has at least eightdwarf planets:Ceres,Pluto,Haumea,Q

In [ ]:
context = text
question = "The Solar System was formed?"

# Use the pipeline to answer the question
result = nlp(question=question, context=context, padding="max_length", truncation=True)

print("Answer:", result['answer'])

Answer: 4.568 billion years ago


##Gradio

In [ ]:
import gradio as gr
import pandas as pd
import plotly.express as px
pd.set_option('display.max_colwidth', 0)



def answer(input,url):
    text = extract_text_from_url(url)
    context = clean_text_basic(text)

    result = nlp(question=input, context=context, padding="max_length", truncation=True)
    return result['answer']

with  gr.Blocks() as iface:
    md = gr.Markdown(
      """
      # Hello World!
      Find the data in your url. =))
      """)
    url = gr.Textbox(placeholder="Please input your url to crawl data.",label="Url")
    inp = gr.Textbox(placeholder="Please input your questions.",label="Question")

    btn = gr.Button("Submit")
    out = gr.Textbox(placeholder="Your answer here.",label="Answer")
    btn.click(fn=answer, inputs=[inp,url], outputs=out)

iface.launch()




Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0bac8a607a35e89c97.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
